This notebook is a handler to inspect GeoCODES schema.org metadata, conforming to ESIP Science On Schema.org conventions
Notebook is set up with these parameters from the GeoCODES UI: 
    url-- the contentURL from the distribution
    ext -- the schema:encodingFormat from the distribution metadata; defaults to NONE if no value passed,
First acquire the distribution file and checking that it can be read, 


In [19]:
#github.com/nteract/papermill'parameters'tag used to inject them into template then post a gist runable by colab
url,ext,urn=None,None,None

In [1]:
# Parameters
# these are the parameters passed from the GeoCodes Searth interface
# the schema:distribution download URL. This should be a content URL

url = "https://www.hydroshare.org/hsapi/resource/b541f44606a44a4a911e0e09d1b88d74"
ext = "application/zip"
urn = "urn:gleaner:milled:hydroshare:9ebedd91635b1a39160afcac44ecf3ccd2d22794"

In [2]:
#!pip install httpimport

In [3]:
import httpimport
import requests
import pandas
with httpimport.github_repo('earthcube', 'earthcube_utilities'):
    import earthcube_utilities as ec

ec.wget_rdf(urn)
#!ls -l   or FilePaneBrowser on left, to see it
df=ec.read_file(url,ext)
print (f'df={df}')

response = requests.get(url)
tcode = response.status_code

theheaders = response.headers['content-type']

formatMatch = False
if theheaders != None and ext != None and theheaders in ext:
    formatMatch = True


print (f'response status={tcode}, headers: {theheaders}, formatmatch: {formatMatch}')

df=can not read zip w/o knowing what is in it, doing:[!wget $url ],to see:[ !ls -l ]size:None or FileExplorerPane on the left[Warn:No File]
response status=200, headers: application/zip, formatmatch: True


In [4]:
# **************** DISPATCH ****************

In [5]:
# ****************** ZIP ARCHIVE ******************
#anything that has a response content header for a zip archive
# TBD look at ext for specific kinds of zip archives that can be processed in more interesting ways
if 'application/zip' in theheaders:
  import zipfile   
  with open('test.zip', 'wb') as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)
    fd.close
  with zipfile.ZipFile('test.zip','r') as myzip:
    thefiles = myzip.namelist()
  print ('Contents of ZIP archive: \n')
  for fname in thefiles:
    print (ec.path_leaf(fname))

Contents of ZIP archive: 

mainriversites.dbf
mainriversites_meta.xml
mainriversites.prj
mainriversites_resmap.xml
mainriversites.sbn
mainriversites.sbx
mainriversites.shp
mainriversites.shx
resourcemap.xml
resourcemetadata.xml
bagit.txt
manifest-md5.txt
readme.txt
tagmanifest-md5.txt


In [10]:
# **************************** MagIC tsv database file *********************
#do something with MagIC tsv database files

if 'earthref-tsv' in ext.lower():
    print ('MagIC file')
    import operations.magic.MagicToDataFrames as magic
    resultframedict = magic.magicToDataframe(txt=response.text)
    
    if not(resultframedict['contribution'].empty):
        thecontribs=resultframedict['contribution']
        thekeys=(list(thecontribs))
        for index, row in thecontribs.iterrows():
            if 'contributor' in thekeys:
                contrib = 'contributor: ' + row['contributor'] + '; \n'
            else:
                contrib = ''
            if 'author' in thekeys:
                auth = 'author: ' + row['author'] + '; \n'
            else:
                auth = ''

            if 'reference' in thekeys:
                refer = 'reference: ' + row['reference'] + '; \n'
            else:
                refer = ''

            if 'data_model_version' in thekeys:
                dm = 'data_model_version: ' + row['data_model_version'] + ';'
            else:
                dm = ''

            print(contrib  + auth  +  refer  + dm)


In [11]:
# ********************** NetCDF File **********************

In [16]:
thegraph=ec.wget_rdf(urn)
print (urn)
print (thegraph)

urn:gleaner:milled:hydroshare:57d631b8ca18b4378ad4db0b9271ebca88311b53
None


In [17]:
ec.viz(thegraph) #shows .nt metadata from urn, can use for (rdf)xml as well 

TypeError: expected str, bytes or os.PathLike object, not NoneType